In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import os
import time
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
from pydrake.all import (
    StartMeshcat,
    CollisionCheckerParams,
    RobotDiagramBuilder,
    MeshcatVisualizer,
    Parser,
    LoadModelDirectives,
    ProcessModelDirectives,
    SceneGraphCollisionChecker,
    AutoDiffXd,
    RigidTransform_,
    IrisNp2Options,
    IrisZoOptions,
    SnoptSolver,
    IpoptSolver,
    IrisParameterizationFunction,
    MathematicalProgram,
    HPolyhedron,
    IrisNp2,
    IrisZo,
    Hyperellipsoid,
    RandomGenerator,
    ComputePairwiseIntersections,
    GcsTrajectoryOptimization,
    Point,
    GraphOfConvexSetsOptions,
    FunctionHandleTrajectory,
    InitializeAutoDiff,
    ExtractGradient,
    Toppra,
    PathParameterizedTrajectory,
    PiecewisePolynomial,
    CompositeTrajectory,
    CalcGridPointsOptions,
    BsplineBasis,
    BsplineTrajectory,
    KinematicTrajectoryOptimization,
    MinimumDistanceLowerBoundConstraint,
    PyFunctionConstraint,
    SolverOptions,
    CommonSolverOption,
    Solve,
)

In [ ]:
import src.iiwa_analytic_ik as iiwa_analytic_ik
import src.common as common
import src.rrt as rrt
import src.shortcut as shortcut

# Parameters

In [ ]:
directives_file = os.path.join(common.RepoDir(), "models/old_shelves.dmd.yaml")
grasp_distance = 0.6
GC2 = 1
GC4 = 1
GC6 = -1
q_tilde_bottom = np.array([-0.6430910102907225, 1.9156121024586796, -1.7968254667817805, 1.2945447141185198, -0.023834531305537934, -0.876966810663043, -1.7041643160834519, 1.45])
q_tilde_middle = np.array([-0.5997312520566763, 1.489780849654964, -1.4739679827359913, 1.2905366081785483, -0.04421061906813227, -0.8793712572715165, -1.1603461715511334, 1.45])
q_tilde_top = np.array([-0.1994994216078726, 0.9140739951190965, -2.236618320862171, 0.5238879195899456, 0.7998441913611017, -1.3575398006936048, -1.0153092816310436, 2.41])

In [ ]:
seeds = [
    q_tilde_bottom,
    np.array([-0.7341522021700233, 1.9192492722970935, -1.849050540687353, 1.4690188979347225, -0.022913995470214974, -0.7839567180379224, -1.735834076048031, 1.45]),
    np.array([-0.816394667473979, 1.9228828117510568, -1.9042766014076622, 1.6254903325102958, -0.020884458583263387, -0.6994788210824544, -1.773950224396859, 1.45]),
    np.array([-0.9076736984240236, 1.7999568628541147, -1.8278258357789336, 1.8976493299850326, -0.032028511314404574, -0.5492230012012871, -1.624933169711267, 1.45]),
    np.array([-0.90780384835653, 1.5443282072400564, -1.480882097408486, 1.9741581801564516, -0.07059018895327443, -0.5065618808846135, -1.1610690777465094, 1.45]),
    np.array([-0.877792385473089, 1.283945692440691, -1.1673903163525974, 1.7986279782674526, -0.08798686286997325, -0.605914842625335, -0.7496023024205761, 1.45]),
    np.array([-0.7363360141869535, 1.0835790623705088, -1.102219288049605, 1.3727471630916555, -0.07210415656873102, -0.8362237374759414, -0.6008766712030682, 1.45]),
    np.array([-0.7093225760311644, 0.8650840325295542, -1.4794100092984808, 1.2099934253928784, 0.44173726212402287, -0.9673197772349095, -0.9450827150678346, 2.0]),
    np.array([-0.5237049267440886, 0.7086764066165658, -1.9872212610757156, 1.045742737284787, 0.8594286107005795, -1.171705603794283, -1.1435157398017397, 2.41]),
    np.array([-0.37540312953312194, 0.7958305227244739, -2.112215906760149, 0.8433434932970723, 0.8316630398644385, -1.2430896040746857, -1.1077155278001196, 2.41]),
    q_tilde_top,
    np.array([-0.7686406052800139, 1.504938625148829, -1.4584578152597332, 1.655937158932382, -0.055175677810583384, -0.6834840454669682, -1.1418310479792013, 1.45]),
    q_tilde_middle,
]

# Set Up Environment

In [ ]:
meshcat = StartMeshcat()

In [ ]:
params = CollisionCheckerParams()
builder = RobotDiagramBuilder(time_step=0.0)
MeshcatVisualizer.AddToBuilder(builder.builder(), builder.scene_graph(), meshcat)
plant = builder.plant()

parser = Parser(plant)
package_xml_path = os.path.join(common.RepoDir(), "package.xml")
parser.package_map().AddPackageXml(package_xml_path)
directives = LoadModelDirectives(directives_file)
ProcessModelDirectives(directives, parser)

params.robot_model_instances = [
    plant.GetModelInstanceByName("iiwa_left"),
    plant.GetModelInstanceByName("iiwa_right")
]

plant.Finalize()
diagram = builder.Build()

params.model = diagram
params.edge_step_size = 0.01
checker = SceneGraphCollisionChecker(params)

In [ ]:
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

# Build Regions

## Set Up the Parametrization

In [ ]:
analytic_ik = iiwa_analytic_ik.Analytic_IK_7DoF(iiwa_analytic_ik.iiwa_alpha,
                                                iiwa_analytic_ik.iiwa_d,
                                                iiwa_analytic_ik.iiwa_limits_lower,
                                                iiwa_analytic_ik.iiwa_limits_upper)

def q_to_ee_target(q):
    global grasp_distance

    ad = isinstance(q[0], AutoDiffXd)
    T = AutoDiffXd if ad else float
    
    tf_goal = analytic_ik.FK(q)
    ang = (180 - 2. * 68.) * np.pi / 180.
    c, s = np.cos(ang), np.sin(ang)
    tf_goal[:-1,:-1] = tf_goal[:-1,:-1] @ np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]]) @ np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    tf_goal[:-1,-1] = tf_goal[:-1,-1] + tf_goal[:-1,:-1] @ np.array([0, 0, -grasp_distance])
    tf_goal[:-1,-1] = tf_goal[:-1,-1] + np.array([0, -0.765, 0])
    return RigidTransform_[T](tf_goal)

def parameterization(q_tilde):
    global GC2, GC4, GC6
    q_full = np.zeros(14, dtype=type(q_tilde[0]))
    q_full[:7] = q_tilde[:7]
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    q_follower = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi)
    q_full[7:] = q_follower
    return q_full

In [ ]:
iris_zo_options = IrisZoOptions()
iris_np2_options = IrisNp2Options()
for iris_options in [iris_zo_options, iris_np2_options]:
    iris_options.parameterization = IrisParameterizationFunction(parameterization, 8)

    iris_options.sampled_iris_options.verbose = True
    iris_options.sampled_iris_options.max_iterations = 1
    iris_options.sampled_iris_options.relax_margin = True

    iris_options.sampled_iris_options.delta = 0.01
    iris_options.sampled_iris_options.epsilon = 0.01
    
#     iris_options.sampled_iris_options.sample_particles_in_parallel = True

iris_np2_options.add_hyperplane_if_solve_fails = True
iris_np2_options.solver_options.SetOption(SnoptSolver().solver_id(), "Major iterations limit", 50)

iris_prog = MathematicalProgram()
q_tilde_vars = iris_prog.NewContinuousVariables(8, "q_tilde")
for iris_options in [iris_zo_options, iris_np2_options]:
    iris_options.sampled_iris_options.prog_with_additional_constraints = iris_prog

## Visualize the Key Configurations

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, parameterization(q_tilde_bottom))
diagram.ForcedPublish(context)

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, parameterization(q_tilde_middle))
diagram.ForcedPublish(context)

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, parameterization(q_tilde_top))
diagram.ForcedPublish(context)

In [ ]:
idx = 4

context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, parameterization(seeds[idx]))
diagram.ForcedPublish(context)

## Set Up the Additional Constraints

In [ ]:
# This constraint enforces reachability.
def unclipped_vals(q_tilde):
    global GC2, GC4, GC6
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    unclipped_vals = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi, return_unclipped_vals=True)
    return unclipped_vals

iris_prog.AddConstraint(unclipped_vals,
                        -np.ones(4),
                        np.ones(4),
                        q_tilde_vars)

In [ ]:
# This constraint enforces the subordinate arm's joint limits.
def subordinate_arm_config(q_tilde):
    global GC2, GC4, GC6
    tf_goal = q_to_ee_target(q_tilde[:7])
    psi = q_tilde[7]
    q_follower = analytic_ik.IK(tf_goal, [GC2, GC4, GC6], psi)
    return q_follower

iris_prog.AddConstraint(subordinate_arm_config,
                        iiwa_analytic_ik.iiwa_limits_lower,
                        iiwa_analytic_ik.iiwa_limits_upper,
                        q_tilde_vars)

In [ ]:
# The self-motion parameter is circle-valued (wraps around at 2pi), but we ignore that for now.
domain_lower = np.hstack((iiwa_analytic_ik.iiwa_limits_lower, [0.0]))
domain_upper = np.hstack((iiwa_analytic_ik.iiwa_limits_upper, [2.0 * np.pi]))
domain = HPolyhedron.MakeBox(domain_lower, domain_upper)

## Grow the Regions

In [ ]:
def grow_region(seed):
    # Grow collision free region
    region = IrisNp2(checker,
                     Hyperellipsoid.MakeHypersphere(1e-2,
                                                    seed),
                     domain,
                     iris_np2_options)
    return region

In [ ]:
regions = []
for i in tqdm(range(len(seeds))):
    regions.append(grow_region(seeds[i]))
print("Average number of faces:", np.mean([len(region.b()) for region in regions]))

## Check Connectivity

In [ ]:
intersections, _ = ComputePairwiseIntersections(regions, [])
mat = np.zeros((len(regions), len(regions)))
if intersections:
    rows, cols = zip(*intersections)
    mat[rows, cols] = 1

plt.imshow(mat)
plt.show()

print("Number of connected components", nx.number_connected_components(nx.from_numpy_array(mat)))

## Visualize a Walk around a Region

In [ ]:
# context = diagram.CreateDefaultContext()
# plant_context = plant.GetMyContextFromRoot(context)

# region = regions[0]
# q_tilde = seeds[0].copy()
# rng = RandomGenerator()
# np.random.seed(0)
# for _ in range(20):
#     q_tilde_next = region.UniformSample(rng)
#     for t in np.linspace(0, 1, 50):
#         q = parameterization(t * q_tilde_next  + (1 - t) * q_tilde)
#         plant.SetPositions(plant_context, q)
#         diagram.ForcedPublish(context)
#         time.sleep(0.02)
#     q_tilde = q_tilde_next.copy()

# Plan with GcsTrajectoryOptimization

In [ ]:
start = q_tilde_bottom.copy()
goal = q_tilde_middle.copy()

## Actually Do the Planning

In [ ]:
order = 2
continuity = 1

gcs = GcsTrajectoryOptimization(8)
if continuity >= 1:
    for continuity_order in range(1, continuity + 1):
        gcs.AddPathContinuityConstraints(continuity_order)

main_graph = gcs.AddRegions(regions, order, h_min=0.1, h_max=100, name="")
start_graph = gcs.AddRegions([Point(start)], 0)
goal_graph = gcs.AddRegions([Point(goal)], 0)
gcs.AddEdges(start_graph, main_graph)
gcs.AddEdges(main_graph, goal_graph)

# gcs.AddPathEnergyCost()
gcs.AddPathLengthCost()
gcs.AddTimeCost()
gcs.AddVelocityBounds(-np.ones(8), np.ones(8))

options = GraphOfConvexSetsOptions()
options.max_rounding_trials = 100
options.max_rounded_paths = 100
options.convex_relaxation = True

gcs_traj, result = gcs.SolvePath(start_graph, goal_graph, options)
print(result.is_success())

# Comparison: RRT-Connect

In [ ]:
def RandomConfig():
    return np.random.uniform(low=domain_lower, high=domain_upper)

def ValidityChecker(q_tilde):
    if np.any(unclipped_vals(q_tilde) > np.ones(4)):
        return False
    if np.any(unclipped_vals(q_tilde) < -np.ones(4)):
        return False
    
    q_full = parameterization(q_tilde)
    q_sub = q_full[7:]
    if np.any(q_sub < iiwa_analytic_ik.iiwa_limits_lower):
        return False
    if np.any(q_sub > iiwa_analytic_ik.iiwa_limits_upper):
        return False
    
    if np.any(np.abs(q_sub[[1, 3, 5]]) < 1e-2):
        return False
    
    return checker.CheckConfigCollisionFree(q_full)

In [ ]:
rrt_options = rrt.RRTOptions(
    step_size = 2e-1,
    check_size = 1e-2,
    max_vertices = 1e4,
    max_iters = 1e6,
    goal_sample_frequency = 0.01,
    always_swap = False
)
rrt_planner = rrt.BiRRT(RandomConfig, ValidityChecker)

np.random.seed(0)
path = rrt_planner.plan(start, goal, rrt_options)

In [ ]:
np.random.seed(0)
shortcut_path = shortcut.shortcut(path.copy(), ValidityChecker, num_tries=1e2, check_size=rrt_options.check_size)

In [ ]:
rrt_traj_segments = [
    PiecewisePolynomial.CubicWithContinuousSecondDerivatives(
        np.array([float(i) - 1.0, float(i)]),
        np.array([shortcut_path[i-1], shortcut_path[i]]).T,
        np.zeros(8),
        np.zeros(8)
    )
    for i in range(1, len(shortcut_path))
]
rrt_traj = CompositeTrajectory(rrt_traj_segments)

## (Optional) Grow Regions along RRT Path

In [ ]:
# for i in tqdm(range(1, len(shortcut_path))):
#     q_tilde_0 = shortcut_path[i-1]
#     q_tilde_1 = shortcut_path[i]
#     if regions:
#         skip = False
#         for idx, region in enumerate(regions):
#             if region.PointInSet(q_tilde_1):
#                 print("Point is in region %d, skipping" % idx)
#                 skip = True
#                 break
#         if skip:
#             continue
#     for iris_options in [iris_zo_options, iris_np2_options]:
#         iris_options.sampled_iris_options.containment_points = np.array([
#             q_tilde_0,
#             q_tilde_1,
#         ]).T
#     q_tilde = (q_tilde_0 + q_tilde_1) / 2
# #     regions.append(IrisZo(checker, Hyperellipsoid.MakeHypersphere(1e-2, q_tilde), domain, iris_zo_options))
#     regions.append(IrisNp2(checker, Hyperellipsoid.MakeHypersphere(1e-2, q_tilde), domain, iris_np2_options))

# Comparison: Kinematic Trajectory Optimization
(Initialized from the RRT-Connect Solution)

In [ ]:
spline_order = 4
t0 = rrt_traj.start_time()
t1 = rrt_traj.end_time()

In [ ]:
# Option 1: approximately follow the PL trajectory by directly using the vertices as the knot points.
# This doesn't guarantee feasibility, but is generally easier for the optimizer to improve past the initial guess.
control_points_matrix = np.array([
    point
    for point in shortcut_path
]).T

In [ ]:
# Option 2: exactly follow the PL trajectory by duplicating control points.
# This better guarantees feasibility, but is harder for the optimizer to improve past the initial guess.
# control_points_matrix = np.array([
#     point
#     for point in shortcut_path
#     for _ in range(spline_order - 1)
# ]).T

In [ ]:
# Construct the initial guess, and the trajectory optimization object
basis = BsplineBasis(spline_order,
                     control_points_matrix.shape[1],
                     initial_parameter_value=t0,
                     final_parameter_value=t1)
initial_traj = BsplineTrajectory(basis, control_points_matrix)

trajopt = KinematicTrajectoryOptimization(initial_traj)

In [ ]:
# Start and end points
trajopt.AddPathPositionConstraint(initial_traj.value(initial_traj.start_time()),
                                  initial_traj.value(initial_traj.start_time()),
                                  0)
trajopt.AddPathPositionConstraint(initial_traj.value(initial_traj.end_time()),
                                  initial_traj.value(initial_traj.end_time()),
                                  1)
pass

In [ ]:
# Define collision-free constraints
minimum_distance = 0.001 # 1mm
influence_distance = 0.05 # 5cm
minimum_distance_constraint_diagram_context = diagram.CreateDefaultContext()
minimum_distance_constraint_plant_context = plant.GetMyContextFromRoot(minimum_distance_constraint_diagram_context)
minimum_distance_constraint = MinimumDistanceLowerBoundConstraint(plant,
                                                                  minimum_distance,
                                                                  minimum_distance_constraint_plant_context,
                                                                  None,
                                                                  influence_distance - minimum_distance)

def parametrized_minimum_distance_callable(q_tilde):
    q_full = parameterization(q_tilde)
    return minimum_distance_constraint.Eval(q_full)

parametrized_minimum_distance_constraint = PyFunctionConstraint(8,
                                                                parametrized_minimum_distance_callable,
                                                                [-np.inf],
                                                                [1.0],
                                                                "parametrized_minimum_distance_constraint")

In [ ]:
# Define subordinate arm joint limit constraints
subordinate_arm_joint_limit_constraint = PyFunctionConstraint(8,
                                                              subordinate_arm_config,
                                                              iiwa_analytic_ik.iiwa_limits_lower,
                                                              iiwa_analytic_ik.iiwa_limits_upper,
                                                              "subordinate_arm_joint_limit_constraint")

In [ ]:
# Define reachability constraints
reachability_constraint = PyFunctionConstraint(8,
                                               unclipped_vals,
                                               -np.ones(4),
                                               np.ones(4),
                                               "reachability_constraint")

In [ ]:
# Add the constraints
num_constraints_to_apply = 100
for s in np.linspace(0, 1, num_constraints_to_apply):
    trajopt.AddPathPositionConstraint(parametrized_minimum_distance_constraint, s)
    trajopt.AddPathPositionConstraint(subordinate_arm_joint_limit_constraint, s)
    trajopt.AddPathPositionConstraint(reachability_constraint, s)

In [ ]:
# Add parametrized path energy cost
trajopt.AddPathEnergyCost()
# trajopt.AddPathLengthCost()
pass

In [ ]:
# Check constraint satisfaction at initial guess
satisfied = True
for binding in trajopt.prog().GetAllConstraints():
    c = binding.evaluator()
    val = c.Eval(trajopt.prog().GetInitialGuess(binding.variables()))
    lb, ub = c.lower_bound(), c.upper_bound()
    ok = np.all(val >= lb - 1e-8) and np.all(val <= ub + 1e-8)
    if not ok:
        print(f"{c.__class__.__name__}: value={val}, bounds=[{lb}, {ub}], ok={ok}")
    satisfied &= ok

print("\nAll constraints satisfied at initial guess?", satisfied)

In [ ]:
trajopt_options = SolverOptions()

In [ ]:
# IPOPT Settings
trajopt_options.SetOption(CommonSolverOption.kPrintToConsole, True)
trajopt_options.SetOption(IpoptSolver().solver_id(), "print_level", 5)
trajopt_options.SetOption(IpoptSolver().solver_id(), "max_wall_time", 60)

# Accept if feasible and cost decreases by less than 1%, 5 iterations in a row.
trajopt_options.SetOption(IpoptSolver().solver_id(), "acceptable_tol", 1e-2)
trajopt_options.SetOption(IpoptSolver().solver_id(), "acceptable_iter", 5)
trajopt_options.SetOption(IpoptSolver().solver_id(), "acceptable_constr_viol_tol", 1e-6)

# This option tries to prevent IPOPT from relaxing the constraint bounds very much.
trajopt_options.SetOption(IpoptSolver().solver_id(), "bound_relax_factor", 1e-12)

# Project the final solution onto the original bounds. (Unclear how essential this is.)
trajopt_options.SetOption(IpoptSolver().solver_id(), "honor_original_bounds", "yes")

# This option strongly encourages IPOPT to maintain feasibility, at the cost of worse globalization.
trajopt_options.SetOption(IpoptSolver().solver_id(), "line_search_method", "penalty")

In [ ]:
# SNOPT Settings
# TODO

In [ ]:
solver = IpoptSolver()
# solver = SnoptSolver()

result = solver.Solve(trajopt.prog(), None, trajopt_options)

In [ ]:
print(result.is_success())
trajopt_traj = trajopt.ReconstructTrajectory(result)
print(trajopt_traj.start_time(), trajopt_traj.end_time())
plt.plot(trajopt_traj.vector_values(np.linspace(trajopt_traj.start_time(), trajopt_traj.end_time(), 100)).T)
plt.show()

## Remap and Retime Trajectory

In [ ]:
# traj_to_show = GcsTrajectoryOptimization.NormalizeSegmentTimes(gcs_traj)
# traj_to_show = rrt_traj.Clone()
traj_to_show = trajopt_traj.Clone()

In [ ]:
traj_function = lambda t : parameterization(traj_to_show.value(t).flatten())
full_traj = FunctionHandleTrajectory(traj_function, 14, 1, traj_to_show.start_time(), traj_to_show.end_time())

In [ ]:
def full_traj_derivative(t, order, dt=1e-6):
    """
    Compute trajectory derivatives using finite differences.
    
    Args:
        t: time at which to evaluate.
        order: 1 for first derivative, 2 for second derivative.
        dt: small finite difference step for second derivative.
        
    Returns:
        dydt: (14,1) derivative vector.
    """
    if order == 1:
        # First derivative: use original AutoDiff approach
        x_val = traj_to_show.value(t).flatten()           # (8,)
        xdot_val = traj_to_show.EvalDerivative(t, 1).flatten()  # (8,)

        x_ad = InitializeAutoDiff(x_val).flatten()  # AutoDiff vector
        y_ad = parameterization(x_ad)  # (14,)
        J = ExtractGradient(y_ad)  # (14x8)
        dydt = J @ xdot_val.reshape(-1, 1)  # (14,1)
        return dydt

    elif order == 2:
        return second_derivative_stencil(t, dt)

    else:
        raise RuntimeError(f"Only first and second derivatives supported (requested order={order})")

def second_derivative_stencil(t, dt=1e-6):
    """
    Compute second derivative using a 5-point central difference (more stable).
    """
    f0 = full_traj_derivative(t, order=1)
    f1 = full_traj_derivative(t + dt, order=1)
    f2 = full_traj_derivative(t - dt, order=1)
    f3 = full_traj_derivative(t + 2*dt, order=1)
    f4 = full_traj_derivative(t - 2*dt, order=1)

    d2y_dt2 = (-f3 + 16*f1 - 30*f0 + 16*f2 - f4) / (12 * dt**2)
    return d2y_dt2

In [ ]:
full_traj.set_derivative(full_traj_derivative)

In [ ]:
# Check derivatives

def finite_difference_derivative(traj_function, t, eps=1e-6):
    """Return dy/dt via centered finite differencing."""
    y_plus = traj_function(t + eps)
    y_minus = traj_function(t - eps)
    dydt_fd = (y_plus - y_minus) / (2 * eps)
    return dydt_fd.reshape(-1)

t_test = (full_traj.start_time() + full_traj.end_time()) / 2

y = full_traj.value(t_test).flatten()
ydot_analytic = full_traj.EvalDerivative(t_test, 1).flatten()
ydot_fd = finite_difference_derivative(traj_function, t_test)

diff = ydot_analytic - ydot_fd
rel_err = np.linalg.norm(diff) / np.linalg.norm(ydot_fd)

print("y(t):", y)
print("Analytic ẏ(t):", ydot_analytic)
print("FD approx ẏ(t):", ydot_fd)
print("‖error‖ =", np.linalg.norm(diff))
print("‖error‖ / ‖ẏ_fd‖ =", rel_err)

In [ ]:
# n_segments = traj_to_show.get_number_of_segments()
# n_gridpoints = 30 * n_segments + 1
# gridpoints = np.linspace(full_traj.start_time(), full_traj.end_time(), n_gridpoints)

gridpoints = Toppra.CalcGridPoints(full_traj, CalcGridPointsOptions(max_iter=2, min_points=200))
print(len(gridpoints))

# Add extra gridpoints on the boundary of each trajectory segment.
# n_segments = traj_to_show.get_number_of_segments()
# extra_gridpoints = np.arange(n_segments + 1, dtype=float)

# print(full_traj.end_time())

# gridpoints = np.unique(np.concatenate((gridpoints, extra_gridpoints)))
# print(gridpoints)
# print(len(gridpoints))

In [ ]:
toppra = Toppra(full_traj, plant, gridpoints)
toppra.AddJointVelocityLimit(plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits())
toppra.AddJointAccelerationLimit(plant.GetAccelerationLowerLimits(), plant.GetAccelerationUpperLimits())
toppra.AddJointTorqueLimit(plant.GetEffortLowerLimits(), plant.GetEffortUpperLimits())
time_traj = toppra.SolvePathParameterization()

In [ ]:
retimed_full_traj = PathParameterizedTrajectory(full_traj, time_traj)
print("Trajectory duration:", (retimed_full_traj.end_time() - retimed_full_traj.start_time()))
# If this is `inf`, then try setting the `max_iter` or `min_points` options higher in Toppra.CalcGridPoints

## Visualize the Plan

In [ ]:
qs = [retimed_full_traj.value(t).flatten() for t in np.linspace(retimed_full_traj.start_time(), retimed_full_traj.end_time(), 1000)]
plt.figure(figsize=(20, 12)) # Creates a figure with 10 inches width and 6 inches height
plt.plot(np.linspace(retimed_full_traj.start_time(), retimed_full_traj.end_time(), 1000), qs)
plt.show()

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

frame_delay = 1.0 / 60
for t in np.arange(retimed_full_traj.start_time(), retimed_full_traj.end_time(), frame_delay):
    q = retimed_full_traj.value(t).flatten()
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(context)
    time.sleep(frame_delay)